In this demo, we are going to use LangGraph to create a simple chat agent using the so called "prebuilt agents" framework.  We'll start with the LangGraph code to create and run the agent without Mindlytics. After that runs, we'll integrate Mindlytics to instrument the application to get analytics.  

For this demo, in addition to an `OPENAI_API_KEY`, you will need a Mindlytics api key and project id.  These can be sent in environment variables; `MLSDK_API_KEY` and `MLSDK_PROJECT_ID`.  To run this notebook from the command line, you can

```sh
OPENAI_API_KEY=xxx MLSDK_API_KEY=yyy MLSDK_PROJECT_ID=zzz poetry run jupyter lab examples/langchain/chatbot.ipynb
```

Lets start out with the required imports and checks for environment variables.

In [ ]:
import asyncio
import os
from typing import List
from langchain_core.messages import AIMessageChunk
from langgraph.prebuilt import create_react_agent
from langchain_core.tools import tool
from langgraph.checkpoint.memory import InMemorySaver  # for chat history
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnableConfig
import json
import rich

if not os.getenv("OPENAI_API_KEY"):
    print("Error: OPENAI_API_KEY environment variable not set.")
    print("Please set your OpenAI API key:")
    print("export OPENAI_API_KEY='your-api-key-here'")
    sys.exit(1)

if not (os.getenv("MLSDK_API_KEY") and os.getenv("MLSDK_PROJECT_ID") ):
    print("Error: Mindlytics environment is not set up.  The latter part of this demo will not work without them")


Lets create the LLM model, with parameters to capture tokens.

In [ ]:
MODEL = "gpt-4o-mini"

model = ChatOpenAI(
    model=MODEL,
    temperature=0.7,
    model_kwargs={
        "stream_options": {"include_usage": True},
    },
)


We will create a couple of tools for the agent to use.  The first tool can look up user details based on a user's name.  The second tool can look up order details based on an order number.  

In [ ]:
@tool
async def find_user(name: str) -> str:
    """Find user details by name."""

    # simulate a little time
    await asyncio.sleep(2)
    
    # We would do some sort of database lookup here, but for this demo ...
    match = {
        "name": "Princess Leia",
        "email": "princess.leia@alderaan.gal",
        "address": "Princess Leia Organa\nRoyal Palace\nCity of Aldera\nPlanet Alderaan\nCore Worlds, Galactic Republic"
    }

    return json.dumps(match, indent=2)

@tool
async def lookup_order(order: str) -> str:
    """Look up an order by order id."""

    # simulate a little time
    await asyncio.sleep(2)

    # We would do some sort of database lookup here, but for this demo ...
    match = {
        "order": order,
        "purchased_on": "2024-04-25T06:34:54"
    }

    return json.dumps(match, indent=2)
    

Create the agent:

In [ ]:
checkpointer = InMemorySaver() # manage chat history
agent = create_react_agent(
    model=model,
    tools=[find_user, lookup_order],
    checkpointer=checkpointer,
    prompt="You are the customer service agent at the Galactic Toys and Trinkets store.  To process returns, you can search for user information using supplied tools.",
)

Now lets run the agent with a canned set of user input.

In [ ]:
# Our canned user input
user_messages = [
    "I would like to return a hair bun scrunchy.  It is too small for me!",
    "My name sir, is Princess Leia of Alderaan!",
    "Of course I want to proceed!",
    "My order number is 19-BBY.",
    "Never mind.  We're done here!",
    "Goodbye sir.",
]

# This state is required for chat history (thread_id)
config = {
    "configurable": {
        "thread_id": "1",
    }
}

for user_message in user_messages:
    print(f"User: {user_message}")
    print("Assistant: ", end="", flush=True)
    async for chunk in agent.astream(
        {
            "messages": user_message
        },
        stream_mode=["messages"],
        config=config
    ):
        mode = chunk[0]
        if mode == "messages":
            data: AIMessageChunk = chunk[1][0]
            if isinstance(data, AIMessageChunk) and len(data.content) > 0:
                # Stream a token to the screen
                print(data.content, end="", flush=True)
    
    print("\n")


Now we'd like to integrate Mindlytics into this application.  We'd like to capture three things

* User identification - When we learn about the initially anonymous user, identify this user to Mindlytics
* Conversational turns - Report user/assisant turns
* Function calls - Report function calls


Lets import the Mindlytics stuff we will need:

In [ ]:
from mlsdk import Client, Session, MLEvent
from mlsdk.helpers.langgraph import MLPostModellHook

Lets redefine the `find_user` tool so that when a user record is found in the user database, we can identify this user to Mindlytics.  This user will replace the initially anonymous user that began the conversation.

In [ ]:
@tool
async def find_user(
    name: str, 
    config: RunnableConfig # Add LangGraph annotation to get access to Session
) -> str:
    """Find user details by name."""

    # simulate a little time
    await asyncio.sleep(2)
    
    # We would do some sort of database lookup here, but for this demo ...
    match = {
        "name": "Princess Leia",
        "email": "princess.leia@alderaan.gal",
        "address": "Princess Leia Organa\nRoyal Palace\nCity of Aldera\nPlanet Alderaan\nCore Worlds, Galactic Republic"
    }

    # Send the identification to Mindlytics
    session: Session = config["configurable"].get("session")
    if session is not None:
        await session.user_identify(
            id=match["email"],
            traits=match,
        )

    return json.dumps(match, indent=2)


Now lets re-create the agent so it uses the new `find_user` tool, and we will also add the `MLPostModelHook` Mindlytics helper.  This helper will do all of the work to send conversation turns to Mindlytics.  The helper will also track tool use and send that information to Mindlytics as well.

In [ ]:
checkpointer = InMemorySaver() # manage chat history
agent = create_react_agent(
    model=model,
    tools=[find_user, lookup_order],
    checkpointer=checkpointer,
    prompt="You are the customer service agent at the Galactic Toys and Trinkets store.  To process returns, you can search for user information using supplied tools.",
    post_model_hook=MLPostModellHook(model_name=model.model_name),  # ADD THE MINDLYTICS POST HOOK HERE
)

Now we will create the Mindlytics session and invoke the agent. 

In [ ]:
# The client constructor will read MLSDK_API_KEY and MLSDK_PROJECT_ID 
ml_client = Client()

# Capture any Mindlytic communication errors
ml_errors: List[Exception]  = []
async def on_error(error: Exception):
    # Handle Mindlytics errors here
    ml_errors.append(error)

# Capture Mindlytics Events
ml_events: List[MLEvent] = []
async def on_event(event: MLEvent):
    # Handle Mindlytics events here
    ml_events.append(event)

# Create the session.  We don't know who the user is at first, so pass in our device id
# Make a device_id
import uuid
mac = uuid.getnode()
device_id = f"{mac:012x}"

session = ml_client.create_session(
    session_id=str(uuid.uuid4()),
    conversation_id=str(uuid.uuid4()),
    device_id=device_id,
    on_error=on_error,
    on_event=on_event,
)

# This state will be available to function calls, and other parts of the agent framework
config = {
    "configurable": {
        "session": session,
        "thread_id": session.session_id,
    }
}

# Our canned user input
user_messages = [
    "I would like to return a hair bun scrunchy.  It is too small for me!",
    "My name sir, is Princess Leia of Alderaan!",
    "Of course I want to proceed!",
    "My order number is 19-BBY.",
    "Never mind.  We're done here!",
    "Goodbye sir.",
]

for user_message in user_messages:
    print(f"User: {user_message}")
    print("Assistant: ", end="", flush=True)
    async for chunk in agent.astream(
        {
            "messages": user_message
        },
        stream_mode=["messages"],
        config=config
    ):
        mode = chunk[0]
        if mode == "messages":
            data: AIMessageChunk = chunk[1][0]
            if isinstance(data, AIMessageChunk) and len(data.content) > 0:
                # Stream a token to the screen
                print(data.content, end="", flush=True)
                
    print("\n")

await session.end_session()
await session.flush()
print("ALL DONE!")

# The session is complete.  WAIT UNTIL YOU SEE THIS "ALL DONE" TO PROCEED WITH THE DEMO!!!

print(f"We received {len(ml_events)} events and {len(ml_errors)} errors.")
if len(ml_errors) > 0:
    print()
    print("Mindlytics Errors:")
    for error in ml_errors:
        print("------------------------------------------------")
        print(error)


Lets look at a summary of events:

In [ ]:
for event in ml_events:
    print(event.event)
    

You should see some events. You should see a "Session Started" at the beginning and a "Session Ended" at the end. Within the session there should be a "Conversation Started"/"Conversation Ended" pair, and within the conversation you will see a number of events that were captured. Every event contains a lot of detail. Lets take a detailed look at the "Conversation Summary":

In [ ]:
summary = next((event for event in ml_events if event.event == 'Conversation Summary'), None)
if summary is None:
    print("Cannot find the Conversation Summary event!")

for key, value in summary.properties.items():
    print(f"{key}: {value}")


You can see some statistics about the conversation, the summary and the sentiment of the user. Because we tracked token usage, we have a final cost calculation based on the MODEL you used to run the conversation.

If you want to continue this demo by examining some of the other events in detail, here is a helper function to display one or more events by name:

In [ ]:
def show_events(event_name):
    for event in ml_events:
        if event.event == event_name:
            print()
            print(f"Event: {event.event}")
            print(f"  timestamp: {event.timestamp}")
            for prop in ["session_id", "conversation_id", "event_id", "origin_event_id"]:
                if hasattr(event, prop):
                    print(f"  {prop}: {getattr(event, prop)}")
            if hasattr(event, "properties"):
                print("  properties:")
                for key, value in event.properties.items():
                    print(f"    {key}: {value}")
            if hasattr(event, "user_traits"):
                print("  user_traits:")
                for key, value in event.user_traits.items():
                    print(f"    {key}: {value}")


In [ ]:
show_events("Conversation Function")
